In [47]:
# 1. Импортируем библиотеки
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import category_encoders as ce
import functools
from geopy.geocoders import Nominatim
import geopy.distance as distance
import re

In [20]:
# 2. Считываем датафрейм.
path = ("_data.csv")
df = pd.read_csv(path)

In [21]:
path_2 = "../../rent_real_estate/data_loc.csv"
df_2 = pd.read_csv(path_2)

In [4]:
# 3. Устнавливаем опцию отображения всех столбцов.
pd.set_option('display.max_columns', None)

In [22]:
# 4. Оставляем в датафрейме объявление только по Москве.
df = df[df["Адрес"].str.contains("Москва", case=False)]
df.reset_index(inplace=True, drop=True)

In [29]:
# 5. Ищем дубликаты объявлений по описанию.
duplicate_rows = df[df.duplicated(subset='Описание')]
len(duplicate_rows)

281

In [30]:
# 6. Удаляем дубликаты объявлений по описанию.
df = df.drop_duplicates(subset='Описание')
len(df)

19456

In [31]:
# 7. Удаляем лишние столбцы, которые не повлияют или незначительно повлияют на результат.
df.drop(["Unnamed: 0", "Тип", "Телефоны", "Описание", "Площадь комнат, м2", "Дополнительно", "Название ЖК", "Серия дома", "Ссылка на объявление"], inplace=True, axis=1)

In [32]:
# 8. Меняем название столбцов.
rename_map = {'ID  объявления': 'ID', 'Количество комнат': "number_of_rooms", 'Метро': "metro", 'Адрес': "address", 'Площадь, м2': "apartment_square", 
              'Дом': "house", 'Парковка': "parking_lot", 'Можно с детьми/животными': 'children_pets_allowed','Высота потолков, м': 'ceiling_height',
              'Лифт': 'elevator','Мусоропровод': 'garbage_chute', 'Цена': 'price', 'Ремонт': 'decor', 'Балкон': 'balcony', 'Окна': 'windows', 
              'Санузел': 'bathroom'}

df.rename(columns=rename_map, inplace=True)

In [33]:
# 9. Заполняем пропущенные значения
df["number_of_rooms"].fillna("Апартаменты/свободная планировка", inplace=True)
df["metro"].fillna("Нет_данных", inplace=True)
df["parking_lot"].fillna("Отсутствует", inplace=True)

df["ceiling_height"] = df["ceiling_height"].fillna(df["ceiling_height"].mean())
df["elevator"] = df["elevator"].fillna("Нет данных")
df["garbage_chute"] = df["garbage_chute"].fillna("Нет_данных")
df["children_pets_allowed"] = df["children_pets_allowed"].fillna("Нет_данных")

df["decor"].fillna("Нет_данных", inplace=True)
df["balcony"].fillna("Отсутствует", inplace=True)
df["windows"].fillna("Нет_данных", inplace=True)
df["bathroom"].fillna("Нет_данных", inplace=True)

In [34]:
# 10. Напишем функцию определения количества лифтов в доме и ванных комнат в квартире.
def filter_values(lst, st):
    for i in lst:
        if st in i:
            for z in i:
                if z in ["1", "2", "3", "4", "5", "6", "7", "8", "9"]:
                    return int(z)

In [35]:
# 11. Обработаем категории children_pets_allowed, elevator, bathroom, windows, balcony, decor, parking_lot, number_of_rooms.
df["children_yes"] = df["children_pets_allowed"].apply(
    lambda x: 1 if "Можно с детьми" in x else 0
)
df["pets_yes"] = df["children_pets_allowed"].apply(
    lambda x: 1 if "Можно с животными" in x else 0
)
df["children_and_pets_yes"] = df["children_pets_allowed"].apply(
    lambda x: 1 if x == "Можно с детьми, Можно с животными" else 0
)
df["elevator_type"] = df["elevator"].apply(lambda x: len(x.split(",")))
df["elevator_pass"] = df["elevator"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Пасс") == None
    else filter_values(x.split(","), "Пасс")
)
df["elevator_cargo"] = df["elevator"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Груз") == None
    else filter_values(x.split(","), "Груз")
)
df["garbage_chute_ yes"] = df["garbage_chute"].apply(lambda x: 1 if x == "Да" else 0)
df["bathroom_type"] = df["bathroom"].apply(lambda x: len(x.split(",")))
df["bathroom_comb"] = df["bathroom"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Совмещенный") == None
    else filter_values(x.split(","), "Совмещенный")
)
df["bathroom_separate"] = df["bathroom"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Раздельный") == None
    else filter_values(x.split(","), "Раздельный")
)

df["bathroom_separate"] = df["bathroom"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Раздельный") == None
    else filter_values(x.split(","), "Раздельный")
)
df["windows_street"] = df["windows"].apply(lambda x: 1 if "На улицу" in x else 0)
df["windows_yard"] = df["windows"].apply(lambda x: 1 if "Во двор" in x else 0)
df["windows_all"] = df["windows"].apply(lambda x: 1 if "На улицу и двор" in x else 0)

df["balcony_type"] = df["balcony"].apply(
    lambda x: 0 if x == "Отсутствует" else len(x.split(","))
)
df["balcony_bal"] = df["balcony"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Балкон") == None
    else filter_values(x.split(","), "Балкон")
)
df["balcony_loggia"] = df["balcony"].apply(
    lambda x: 0
    if filter_values(x.split(","), "Лоджия") == None
    else filter_values(x.split(","), "Лоджия")
)

df["decor_designer"] = df["decor"].apply(lambda x: 1 if x == "Дизайнерский" else 0)
df["decor_evro"] = df["decor"].apply(lambda x: 1 if x == "Евроремонт" else 0)
df["decor_cosmetic"] = df["decor"].apply(lambda x: 1 if x == "Косметический" else 0)

df["parking_lot_under"] = df["parking_lot"].apply(
    lambda x: 1 if x == "подземная" else 0
)
df["parking_lot_ground"] = df["parking_lot"].apply(
    lambda x: 1 if x == "наземная" else 0
)
df["parking_lot_open"] = df["parking_lot"].apply(lambda x: 1 if x == "открытая" else 0)
df["parking_multi-level"] = df["parking_lot"].apply(
    lambda x: 1 if x == "многоуровневая" else 0
)

df["rooms"] = df["number_of_rooms"].apply(
    lambda x: int(x.split(",")[0]) if len(x.split(",")) > 1 else 1
)

df["isolated_room"] = df["number_of_rooms"].apply(
    lambda x: 1 if "Изолированная" in [i for i in x.replace(" ", "").split(",")] else 0
)

df["adjacent_room"] = df["number_of_rooms"].apply(
    lambda x: 1 if "Смежная" in [i for i in x.replace(" ", "").split(",")] else 0
)

df["adjacent/isolated"] = df["number_of_rooms"].apply(
    lambda x: 1 if "Обаварианта" in [i for i in x.replace(" ", "").split(",")] else 0
)

df["apartment_open_plan"] = df["number_of_rooms"].apply(
    lambda x: 1
    if "Апартаменты/свободнаяпланировка" in [i for i in x.replace(" ", "").split(",")]
    else 0
)

In [36]:
# 12. Напишем функцию получения текущего курса валют.
@functools.lru_cache(
    maxsize=None
)  # Применим декоратор functools.lru_cache для кэширования результатов вызова функции get_exchange_rate, чтобы избежать повторных запросов к веб-сайту.
def get_exchange_rate(currency):
    url = "https://www.cbr.ru/currency_base/daily/"  # Сохраняем URL веб-страницы Центробанка России, где содержится информация о ежедневных курсах валют
    response = requests.get(
        url
    )  # Отправляем HTTP-запрос на указанный URL, чтобы получить содержимое веб-страницы.
    soup = BeautifulSoup(
        response.content, "html.parser"
    )  # Используем библиотеку BeautifulSoup для парсинга (анализа) содержимого HTML веб-страницы.
    for row in soup.find_all(
        "tr"
    ):  # Перебираем все элементы tr (строки) в HTML-таблице на веб-странице.
        cells = row.find_all(
            "td"
        )  # Для каждой строки  находим все ячейки (элементы td) в этой строке.
        if (
            len(cells) > 1 and cells[1].text.strip() == currency
        ):  # Проверяем, что количество ячеек больше 1 (чтобы избежать ошибок) и текст второй ячейки соответствует переданному коду валюты.
            return float(
                cells[4].text.replace(",", ".")
            )  # Если условие выше выполнено, то мы возвращаем курс валюты, который находится в пятой ячейке таблицы (индекс 4), и заменяем запятую на точку для правильного чтения десятичной дроби в Python.

In [37]:
# 13. Напишем функцию конвертации валют.
def convert_price(x):
    try:
        if "руб" in x:
            return float(x.split()[0])
        elif "$" in x:
            return float(x.split()[0]) * get_exchange_rate("USD")
        else:
            return float(x.split()[0]) * get_exchange_rate("EUR")
    except:
        return 0

In [38]:
# 14. Заменяем значения на числовые значения в столбце price.
df["price"] = df["price"].apply(lambda x: convert_price(x))

In [39]:
# 15. Проставляем общую площадь в столбце apartment_square. Меняем тип на float.
df["apartment_square"] = df["apartment_square"].apply(
    lambda x: x.split("/")[0] if isinstance(x, str) else x
)
df["apartment_square"] = df["apartment_square"].astype(float)

In [40]:
# 16. Разделяем столбец house на 2 столбца: floor-nfloors, house_type.
df[["floor-nfloors", "house_type"]] = df["house"].str.split(", ", expand=True)

In [41]:
# 17. Разделяем столбец floor-nfloors на 2 столбца: floor, nfloors. Меняем тип на int.
df[["floor", "nfloors"]] = df["floor-nfloors"].str.split("/", expand=True)

df["floor"] = df["floor"].astype(int)
df["nfloors"] = df["nfloors"].astype(int)

In [42]:
# 18. Проверяем какие значения в столбце house_type.
df["house_type"].unique()

array(['Монолитный', 'Монолитно-кирпичный', None, 'Панельный',
       'Сталинский', 'Кирпичный', 'старый фонд', 'Блочный', 'Деревянный',
       'Щитовой'], dtype=object)

In [43]:
# 19. Заменяем пустые значения.
df["house_type"].fillna("Нет_данных", inplace=True)

In [44]:
# 20. Для кодировки столбца house_type применяем метод Target Encoding,
# который подставит медианное значение стоимости квартиры для соответствующей категории столбца house_type.

# Инициализация Target Encoder.
target_encoder = ce.TargetEncoder(
    cols=["house_type"], min_samples_leaf=10, handle_unknown="value"
)

# Применение Target Encoder к датасету без явного указания цены.
df["house_type_fit_to_price"] = target_encoder.fit_transform(
    df["house_type"], df["price"]
)

In [45]:
# 21. Удаляем оставшиеся столбцы
df.drop(
    [
        "number_of_rooms",
        "house",
        "floor-nfloors",
        "house_type",
        "decor",
        "balcony",
        "windows",
        "bathroom",
        "children_pets_allowed",
        "elevator",
        "garbage_chute",
        "parking_lot",
    ],
    inplace=True,
    axis=1,
)

In [89]:
# Вычисляем количество километров до метро, отсеивая крайние значения
def calculate_distance_to_metro(data: list):
    distance_list = []
    index = []
    for i, distance in enumerate(data):
        if walk := re.match(r".*?\((\d+)\s.*пеш", distance):
            if int(walk[1]) > 40:
                distance_list.append(None)
            else:
                distance_list.append(int(walk[1]) * 0.083)
        elif drive := re.match(r".*?\((\d+)\s.*маш", distance):
            if int(drive[1]) > 40:
                distance_list.append(None)
            else:
                distance_list.append(int(drive[1])  * 0.5)
        else:
            distance_list.append(None)
            index.append(i)
    return distance_list

In [92]:
# Вычисляем расстояние от квартиры до метро и удаляем ненужную колонку
df["metro"].fillna("", inplace=True)
df["distance_to_metro"] = calculate_distance_to_metro(df["metro"])
df["distance_to_metro"].fillna(df["distance_to_metro"].mean(), inplace=True)
df.drop("metro", axis=1, inplace=True)

KeyError: 'metro'

In [ ]:
# 
highway_subset = df[(df["address"].str.contains("ш.") == True) & (df["location"].isna() == True)]
highway_subset['address'] = highway_subset['address'].str.replace(',', '')
list_address = []
for row in highway_subset["address"]:
    list_address.append(row.split("ш.")[0])
highway_subset["address"] = list_address

In [ ]:
def address_sub(addresses):
    modified_addresses = []
    for address in addresses:
        address = re.sub(r"(\d)([А-Я])(к|К)", r'\1\2 \3', address)
        address = re.sub(r"пос|р-н| ао|Северо-Западный|№|уч.|тер.", "", address)
        address = re.sub(r"(улица) (\d+-[яЯ])", r"\2 \1", address)
        address = re.sub(r"кв-л", r"квартал", address)
        modified_addresses.append(address)
    return modified_addresses

In [ ]:
highway_subset["address"] = address_sub(highway_subset["address"])

In [ ]:
address_list = []
def do_geocode(address, list, attempt=1, max_attempts=5):
  try:
    list.append(Nominatim(user_agent="my_request").geocode(address, timeout=None)[-1])
  except GeocoderTimedOut:
    if attempt <= max_attempts:
      return do_geocode(address, attempt=attempt+1)
    raise

In [ ]:
for row in tqdm(df_subset['address']):
    try:
        do_geocode(row, address_list)
    except:
        address_list.append('None')
highway_subset["location"] = address_list
df[(df["address"].str.contains("ш.") == True) & (df["location"].isna() == True)]["location"] == highway_subset["location"]

In [ ]:
df_subset = df[(df["address"].str.contains("ш.") == False) & (df["location"].isna() == True)]

In [ ]:
sub_list
for row in tqdm(df_subset['address']):
    try:
        do_geocode(row, sub_list)
    except:
        address_list.append('None')
df_subset["location"] = sub_list
df[(df["address"].str.contains("ш.") == False) & (df["location"].isna() == True)]["location"] == highway_subset["location"]

In [54]:
df["int_location"] = df["location"].str.split(",")

In [78]:
list_of_locations = []
for item in df["int_location"]:
    a, b = item[0], item[1]
    list_of_locations.append((float(a.strip("(")), float(b.strip(")"))))

In [82]:
df["int_location"] = list_of_locations

In [85]:
center_location = (55.752121, 37.617664)
df[df["location"] == "None"] = np.nan
df.dropna(subset=["location"], axis=0, inplace=True)
df["distance_to_center"] = df["int_location"].apply(lambda x: distance.geodesic(center_location, x).km)

In [86]:
df["distance_to_center"]

0         2.493411
1         2.493411
2         2.493411
3         1.935186
4         1.589391
           ...    
19732    16.161138
19733    18.795418
19734    18.177533
19735    17.398762
19736    17.142200
Name: distance_to_center, Length: 19324, dtype: float64

In [87]:
df.drop(["address", "location", "int_location"], axis=1, inplace=True)

In [91]:
# 22. Выводим финальный датасет в файл.
df.to_csv("data.csv")

In [97]:
# 23. Проверяем типы данных.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19324 entries, 0 to 19736
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       19324 non-null  int64  
 1   apartment_square         19324 non-null  float64
 2   price                    19324 non-null  float64
 3   ceiling_height           19324 non-null  float64
 4   children_yes             19324 non-null  int64  
 5   pets_yes                 19324 non-null  int64  
 6   children_and_pets_yes    19324 non-null  int64  
 7   elevator_type            19324 non-null  int64  
 8   elevator_pass            19324 non-null  int64  
 9   elevator_cargo           19324 non-null  int64  
 10  garbage_chute_ yes       19324 non-null  int64  
 11  bathroom_type            19324 non-null  int64  
 12  bathroom_comb            19324 non-null  int64  
 13  bathroom_separate        19324 non-null  int64  
 14  windows_street           19